In [22]:
import pandas as pd
import re
import os
from pathlib import Path
import logging

# Set Basic configuration of logging
logging.basicConfig(level=logging.INFO)

In [23]:
def run_grade_update():
    """
    Finds grade files using regex, merges iClicker scores,
    and outputs an updated gradebook.
    """
    # Clean up previous output files
    for p in Path.cwd().glob('updated_*'):
        if p.is_file():
            p.unlink()
            logging.info(f"🧹 Cleaned up previous file: {p.name}")

    # --- 1. File Discovery using User-Provided Regex ---
    # This block now correctly finds your files based on their patterns.
    try:
        all_files_in_dir = os.listdir()
        bc_pattern = '.*_Grades-CIVENG_93.csv'
        ic_pattern = 'iClicker_GradesExport_Canvas_.*.csv'

        bc_filename = next(f for f in all_files_in_dir if re.match(bc_pattern, f))
        ic_filename = next(f for f in all_files_in_dir if re.match(ic_pattern, f))

        logging.info(f"✅ Found gradebook by regex: {bc_filename}")
        logging.info(f"✅ Found iClicker file by regex: {ic_filename}")

    except StopIteration:
        logging.warning("❌ Error: Could not find one or both of the required CSV files in the directory.")
        return

    # --- 2. Load Data ---
    # Directly read the files found by the regex match
    bc = pd.read_csv(bc_filename)
    ic = pd.read_csv(ic_filename)
    
    # Remove the "Points Possible" rows which can cause errors
    bc = bc[bc['Student'] != '    Points Possible'].copy()
    ic = ic[ic['Student'] != 'Points Possible'].copy()

    # --- 3. Dynamically Identify Columns ---
    ic_poll_col = next((col for col in ic.columns if re.match(r'Class \d+ - Poll', col)), None)
    if not ic_poll_col:
        logging.warning("❌ Error: No column like 'Class X - Poll' found in the iClicker file.")
        return

    bc_poll_col = next((col for col in bc.columns if col.startswith(ic_poll_col)), None)
    if not bc_poll_col:
        logging.warning(f"❌ Error: Could not find a matching column for '{ic_poll_col}' in the gradebook.")
        return

    #bc_total_col = next((col for col in bc.columns if re.match(r'iClicker \(Total\) \(\d+\)', col)), None)
    bc_total_col = next((col for col in bc.columns if re.match(r'iClicker \(Total\) \(\d+\)', col)), None)
    if not bc_total_col:
        logging.warning("❌ Error: Could not find 'iClicker (Total) Final Score' column in the gradebook.")
        return
    
    all_bc_poll_cols = [col for col in bc.columns if col.startswith('Class') and 'Poll' in col]
    
    logging.info(f"✔️ Identified Columns:\n  - iClicker Poll: '{ic_poll_col}'\n  - Gradebook Poll: '{bc_poll_col}'\n  - Gradebook Total: '{bc_total_col}'")

    # --- 4. Merge, Calculate, and Update (The Efficient Way) ---
    # Select only the needed iClicker columns and merge them into the main gradebook
    ic_subset = ic[['SIS User ID', ic_poll_col]].rename(columns={ic_poll_col: 'New_Score'})
    merged_df = pd.merge(bc, ic_subset, on='SIS User ID', how='left')

    # Convert all relevant columns to numbers to ensure calculations work
    merged_df['New_Score'] = pd.to_numeric(merged_df['New_Score'], errors='coerce').fillna(0)
    for col in all_bc_poll_cols:
         merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').fillna(0)

    logging.info("\n🔄 Updating poll scores with the maximum of old vs. new...")
    # This vectorized operation replaces your entire loop. It's faster and safer.
    merged_df[bc_poll_col] = merged_df[[bc_poll_col, 'New_Score']].max(axis=1)

    logging.info("🔄 Recalculating the total iClicker score by summing all poll columns...")
    merged_df[bc_total_col] = merged_df[all_bc_poll_cols].sum(axis=1)

    # --- 5. Finalize and Save ---
    # Ensure the output has the same columns as the original gradebook
    final_df = merged_df[bc.columns]
    output_filename = f'updated_{bc_filename}'
    final_df.to_csv(output_filename, encoding='utf-8-sig', index=False)

    logging.info(f"\n✨ Done! Updated gradebook saved to '{output_filename}'.")

# Run the main function
if __name__ == "__main__":
    run_grade_update()

INFO:root:✅ Found gradebook by regex: 2025-09-02T1640_Grades-CIVENG_93.csv
INFO:root:✅ Found iClicker file by regex: iClicker_GradesExport_Canvas_09-02-25.csv
INFO:root:✔️ Identified Columns:
  - iClicker Poll: 'Class 1 - Poll'
  - Gradebook Poll: 'Class 1 - Poll (8935383)'
  - Gradebook Total: 'iClicker (Total) (8935426)'
INFO:root:
🔄 Updating poll scores with the maximum of old vs. new...
INFO:root:🔄 Recalculating the total iClicker score by summing all poll columns...
INFO:root:
✨ Done! Updated gradebook saved to 'updated_2025-09-02T1640_Grades-CIVENG_93.csv'.
